# GMxB Trading Grid

Suppose we have some kind of financial instrument, and let's denote it's value by $V(S(t),r,\sigma,t)$
where $S(t)$ is the underlying price (assume it's a stock index), $r$ is the risk-free rate, $\sigma$ is the volatility process for $S(t)$ and $t$ is the current time.  

Dynamic hedging is an approach to replicating the payoff of the instrument based fundamentally on a first order Talyor expansion.  The strategy can be loosley summarized as follows:

1. Calculate the partical derivatives of $V$ w.r.t. it's key inputs.  
2. Enter into a financial derivative transaction with offsetting sensitivities.
3. Repeat steps one and two "frequently" until the instument matures.



## Greeks and dynamic hedging

The first order sensitivies are called "Greeks" and are defined as follows:

"Delta": $\Delta = \frac{\partial V}{\partial S}$   

"Rho":   $\rho = \frac{\partial V}{\partial r}$      

"Vega":  $\Large{\nu} $$= \frac{\partial V}{\partial \sigma}$    

"theta"  $\theta =  \frac{\partial V}{\partial t}$             

Most often, only some of these "Greeks" are actually matched.  When only delta is matched, the approach is called "delta hedging". If rho is matched as well, the strategy is called "delta-rho" hedging.  And naturally, if vega is also hedged, the strategy is called "delta-rho-vega" hedging.

For reasons we won't get into at the moment, theta is almost never explicitly hedged.


For our purposes, let's assume that we are "delta-rho" hedging and that volatility is constant. Our hedge instruments will be things that have "delta" or "rho" and are easy to buy and sell in either positive or negative amounts. This allows us to trade in and out of them in order to "dial" the delta and rho position of the hedge to exactly offset the  delta and rho of the GMxB.


On a given day $t$, let's call the Hedge portfolio $H(S(t),r(t), t)$, our overall portfolio $\Pi$ then consists of $V$ and $H$.  What happens over a short period of time?

$ \Pi(S(t+ \Delta t),r(t+\Delta r),t+ \Delta t)  - \Pi(S(t),r,t) = \frac{\partial V}{\partial S} + \frac{\partial H}{\partial S} + O^2(\left|(\Delta S, \Delta r)\right|)$ 

$= 0 + O^2(\left|(\Delta S, \Delta r)\right|)$ 

With the second eqaulity holding because of how we constructed the portfolio H

So, as long as $\Delta S$ and $\Delta r$ are "small", the gain or loss is close to zero. Which is what is desired.

We can set the portfolio H based on the Greeks derived from overnight Monte Carlo runs.  However, this means that as the market move during the day, our estimtes of the Greeks will drift away from their "true" values.  

## Trading Grid version 1 (Taylor expansion)




In order to evaluate $V(S(t),r(t))$, Monte Carlo simulations need to be run. These simulations give us an estimate of the value $V(S(t),r)$ but no direct information of the partial derivatives of $V$.  However, we can obtain estimates by running the set of Monte Carlo simulations several times over with the initial value changes just slightly.  Let's first suppose that in addition to our base estimate for $V$, we rerun with the equity values $S(t)$ increased just slightly, say to $S(t) + \epsilon_S)$ where $\epsilon_S = .01 \times S$. And then we also  run with  S decreased to $S(t) - \epsilon_S$

Let's denote these two evaluations 

$V_0 = V(S(t), r)$,

$V_+ = V(S(t) + \delta S(t), r)$, and

$V_- = V(S(t) - \delta S(t), r)$

We thus obtain estimates for both the first ans second order partial derivatives of $V$ *vis*

$$\frac{\partial V}{\partial S} \approx \frac{V_+ - V_-}{2\delta S(t)} = \hat{V}_{S}$$

$$\frac{\partial^2 V}{\partial S^2} \approx   \frac{V_+ - 2V_- + V_-}{(\delta S(t))^2} = \hat{V}_{SS}$$

### Intra-day

For convenience, we can denote our previous days' valuations as taking place at time $t$ and the current time to be $t + \tau$

We can estimate our Delta to be 
$$\frac{\partial V(S(t+ \tau),r) }{\partial S} \approx \hat{V}_{S} + \hat{V}_{SS} \times \left(S(t+\tau) - S(t)\right)$$ 

This will work OK so long $S$ has not changed too much (more on this below).  Another problem however is that $S$ ins't the only thing that can change in material way during the day.  In particular, the risk free rate $r$ can change.  

### Adding in rho

We can first add two more valuation runs, where we increase (and decrease) $r(t)$ by some small amount $\epsilon_r$.  Typically, we would take it to be something like $\epsilon_r = .001$ (10 basis points).



We denote these values as follows:

$$V_{r+} = V(S(t), r + \epsilon_r)$$   and

$$V_{r-} = V(S(t), r - \epsilon_r)$$


and obtain estimates

$$V_{r} \approx   \frac{V_{r+} - V_{r-}}{\delta r}  = \hat{V}_{r} $$

$$V_{rr} \approx \frac{V_{r+} - 2V_0+ V_{r-}}{(\delta r)^2} = \hat{V}_{rr} $$



So, now we can adjust our rho hedge appropiatly using

$$\frac{\partial V(S(t+ \tau),r) }{\partial r} \approx \hat{V}_{r} + \hat{V}_{rr} \times \left(r(t+\tau) - r(t)\right)$$ 


### Cross Greeks

The astute observer will note here that there is also a "cross depency" between delta and rho portions of the hedge due the term $$\frac{\partial^2 V}{\partial S\partial r} \neq 0$$

Of course, if $\frac{\partial^2 V}{\partial S\partial r} \approx 0$, maybe we don't care (spoiler: we do).  

In practice, this is often not something that can just be ingnored.  So, we seek an additional estimate $\hat{V}_{Sr}$ and we can then revise our estimates for delta and rho as

$$\frac{\partial V(S(t+ \tau),r) }{\partial S} \approx \hat{V}_{S} + \hat{V}_{SS} \times \left(S(t+\tau) - S(t)\right) + \hat{V}_{Sr} \times (r(t + \tau) - r(t))$$ 

$$\frac{\partial V(S(t+ \tau),r) }{\partial r} \approx \hat{V}_{r} + \hat{V}_{rr} \times \left(r(t+\tau) - r(t)\right) + \hat{V}_{Sr} \times (S(t+\tau) - S(t))$$ 

Obtaining an estimate $\hat{V}_{Sr}$ from shocks is less straight forward.  The most obvious thing to do would be apply 4 additional shocks : ${(+,+) , (+,-), (-,+), (-, -)} and average the correspnding estimates. The question is whether that level of precsision is justified by the additional computational cost (note, we are nearly doubling the number of runs now). 

Now, we will just assume that only a single additional shock is run (-, -).  **This is something can be assessed as part of the project**.

## Another way to think about it

Having obtained our estimates $[\hat{V}_{S},\hat{V}_{SS}, \hat{V}_{r}, \hat{V}_{rr}, \hat{V}_{Sr}]$  we can imagine that we have constucted an estimator function $\hat{V} \approx V$ where

$$ \hat{V}(\delta S,\delta r) = V_0 + \hat{V}_{S}\, \delta S + \frac{1}{2}\hat{V}_{SS}(\delta S)^2  + \hat{V}_{r}\delta r  + \frac{1}{2}\hat{V}_{rr}(\delta r)^2 + \hat{V}_{Sr}\, (\delta S) (\delta r) $$

 and 
 
 $\delta S = S(t+\tau) - S(t)$ and $\delta r = r(t+\tau) - r(t)$

If you diffentiate this function w.r.t $S(t+\tau)$ and $r(t+\tau)$, you see that the "Greeks" of $\hat{V}$ are the same as the above estimates

But now suppose you had some other estimate $\tilde{V}(\delta S, \delta r) \approx V$. What you know about $\tilde{V} $ is
1. $\tilde{V}(0,0) = V(S(t), r(t))$  (The estimate is perfect as of last night)
2. You can evaluate \tilde{V} easily and exactly

Furthermore, you do may or may not have direct knowledge of the derivatives of $\tilde{V}$

You could still readily estimate intra-day Greeks by evaluating $\tilde{V}$ at various points $(\delta S \pm  \epsilon_S, \delta r \pm \epsilon_r)$ that are small perturbations from $(\delta S, \delta r)$

Why would you do this?  Because the Taylor approximation breaks down quickly when $\delta S$ or $\delta r$ get too big.  



## Splines

Equity in particular can be quite volatile whith $S(t)$ changing by as much as $10\%$ within a day. 
One approach for this then is to spend the run time budget obtaining some larger shocks for the equity market. Indeed, this might be a better use of computer resources that multiple delta/rho cross shocks.

To this end, we now assume that we have done the following shocks for our overnight runs:

$S$:  $\pm (1\%, 2\%, 3\%, 5\%, 10\%, 20) $

$r$:  $\pm (.001)$

$(S,r)$:  $ (-1 \%,-.001)$



Along with the base run, we now have run a total of 16 Monte Carlo valuations overnight.

Let us denote these valutions with $V_{su1}, V_{sd1}, V_{su2}, ... , V_{sd20}$ for the pure equity shocks

$V_{ru1}, V_{rd1}$ for the rates, and

$V_{(Sd1,rd1)}$ for the cross shock.



We then can create a spline function $V_{Espline}(\delta S)$ that passes through the points $(V_{su1}, V_{sd1}, V_{su2}, ... , V_{sd20})$ for the approapriate values of $\delta S$

We then can define 

$$\tilde{V}(\delta S, \delta r) \equiv  V_{Espline}(\delta S)  + \hat{V}_{r}\delta r  + \frac{1}{2}\hat{V}_{rr}(\delta r)^2 + \hat{V}_{Sr}\, (\delta S) (\delta r) $$



## The goal

In a nutshell, thie project is about making a better $\tilde{V}$  The tradeoffs being accuracy versus comnputational cost.  The above $\tilde{V}$ is very close to actual practice. But we know it's far from what could be done.  This is where **you** come in.